In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from sqlalchemy import create_engine
%matplotlib inline



In [2]:
power_up = pd.read_csv('ad_power_up.csv')
shop_up = pd.read_csv('ad_super_up_shop.csv')
super_up = pd.read_csv('ad_super_up.csv')
up_plus = pd.read_csv('ad_up_plus.csv')

In [3]:
pu_ctr = pd.read_csv('power_up_ctr.csv')
sus_ctr = pd.read_csv('shop_up_ctr.csv')
su_ctr = pd.read_csv('super_up_ctr.csv')


In [4]:
pu_ctr.columns = ['idx', 'pid', 'view_cnt', 'imp_cnt', 'ctr']
sus_ctr. columns = ['idx', 'pid', 'view_cnt', 'imp_cnt', 'ctr']
su_ctr.columns = ['idx', 'pid', 'view_cnt', 'imp_cnt', 'ctr']

In [5]:
pids = pu_ctr['content_id'].unique().tolist() + sus_ctr['content_id'].unique().tolist() + su_ctr['content_id'].unique().tolist()
pids = list(set(pids))
ids = ','.join('\''+str(i)+'\'' for i in pids)

In [6]:
q = f'''
select id as content_id, left(category_id, 3) as three, left(category_id, 6) as six, category_id
from service1_quicket.product_info
where id in ({ids})
'''
df_ctr = pd.read_sql(q, con=redshift)

In [5]:
# df_ctr.to_csv('sa_contents_info.csv', index=False)
df_ctr = pd.read_csv('sa_contents_info.csv')

In [6]:
df_ctr.columns = ['pid', 'three', 'six', 'category_id']

SA 광고주를 1depth 또는 2depth 카테고리로 구분했을 때, 
- 광고주 수, 
- 광고 컨텐츠 수, 
- 컨텐츠 당 CTR, CPC는 어떠한가?

In [7]:
pu = pd.merge(power_up[['uid', 'pid', 'cpc']], pu_ctr[['pid', 'ctr']], on ='pid')
sus = pd.merge(shop_up[['uid', 'pid', 'cpc']], sus_ctr [['pid', 'ctr']], on='pid')
su = pd.merge(super_up[['uid', 'pid', 'cpc']], su_ctr [['pid', 'ctr']], on='pid')

In [8]:
pu = pd.merge(pu, df_ctr[['pid', 'category_id']], on = 'pid')
sus = pd.merge(sus, df_ctr[['pid', 'category_id']], on = 'pid')
su = pd.merge(su, df_ctr[['pid', 'category_id']], on ='pid')

In [10]:
# sa_ads = pd.merge(pu, sus, on=['uid', 'pid', 'category_id'], how='outer')
sa_ads = pd.merge(sa_ads, su, on=['uid','pid'], how='outer')

In [12]:
one_depth = []
two_depth = []
for i in sa_ads['category_id'].astype(str):
    one_depth.append(i[:3])
    two_depth.append(i[:6])

In [13]:
sa_ads['one_depth'] = one_depth
sa_ads['two_depth'] = two_depth

In [15]:
# sa_ads.fillna(0)
sa_ads['cpc'] = (sa_ads['cpc_x'] + sa_ads['cpc_y'])/2

In [16]:
sa_ads['ctr'] = (sa_ads['ctr_x'] + sa_ads['ctr_y'])/2

In [28]:
sa_ads.groupby('two_depth')['uid'].nunique().to_csv('two_depth_uid.csv')

In [29]:
sa_ads.groupby('two_depth')['pid'].nunique().to_csv('two_depth_pid.csv')

In [30]:
sa_ads.groupby('two_depth')['cpc'].mean().to_csv('two_depth_cpc.csv')
sa_ads.groupby('two_depth')['ctr'].mean().to_csv('two_depth_ctr.csv')